## Import Libraries

In [ ]:
from math import *
import numpy as np
from numpy.linalg import inv

## Define All the Functions

In [ ]:
def eta(P1, P2, xi):
    '''Function used in the Halpin-Tsai relationships.'''
    return ((P1/P2)-1)/((P1/P2)+xi)

def transIsoStiffness(E1, nu1, E2, nu2, V, HT=True):
    '''Return the 2D stiffness tensor of a transversely isotropic composite made
       from materials 1 and 2. Inputs required are the stiffness and Poisson's ratio of
       both materials (E1, nu1, E2, nu2) and volume fraction V of material 1.
       Properties are calculated using Halpin-Tsai relationships by default (HT = True),
       but a Reuss model can be used if desired.'''

    # Calculate Base Material Properties
    G1 = E1/(2*(1+nu1))
    G2 = E2/(2*(1+nu2))

    # Calculate Material Properties
    Ex = E1*V + E2*(1-V) #Voigt

    if HT: #Halpin-Tsai
        Ey = E2*(1 + 2*eta(E1,E2,2)*V)/(1 - eta(E1,E2,2)*V)
        Gxy = G2*(1 + 1*eta(G1,G2,1)*V)/(1 - eta(G1,G2,1)*V)
    else: #Reuss
        Ey = 1/(V/E1+(1-V)/E2)
        Gxy = 1/(V/G1+(1-V)/G2)

    nuxy = nu1*V+nu2*(1-V) #Voigt
    nuyx = nuxy*Ey/Ex #Reciprocal

    # Create Stiffness Matrix
    C = [[Ex/(1-nuxy*nuyx),      nuyx*Ex/(1-nuxy*nuyx), 0],
         [nuyx*Ex/(1-nuxy*nuyx), Ey/(1-nuxy*nuyx),      0],
         [0,                     0,                     Gxy]]

    return np.array(C)

def transIsoCompliance(E1, nu1, E2, nu2, V, HT=True):
    return inv(transIsoStiffness(E1, nu1, E2, nu2, V, HT=True))

def rotaT(theta):
    '''Matrix for stress rotation in 2D.'''
    c = cos(theta)
    s = sin(theta)
    return np.array([[c**2, s**2, 2*s*c],[s**2, c**2, -2*s*c],[-s*c, s*c, c**2-s**2]])

def rotaTP(theta):
    '''Matrix for strain rotation in 2D.'''
    c = cos(theta)
    s = sin(theta)
    return np.array([[c**2, s**2, s*c],[s**2, c**2, -s*c],[-2*s*c, 2*s*c, c**2-s**2]])

def rotateStress(stress, theta):
    '''Rotate 2D stress counterclockwise. Input stress as a 3x1 vector'''
    T = rotaT(theta)
    return T.dot(stress)

def rotateStrain(strain, theta):
    '''Rotate 2D strain counterclockwise. Input  strain as a 3x1 vector'''
    TP = rotaTP(theta)
    return TP.dot(strain)

def rotateStiffness(C, theta):
    '''Rotate the 2D stiffness tensor C counterclockwise by angle theta.'''
    # Define rotation matrix T and T'
    T = rotaT(theta)
    TP = rotaTP(theta)

    # Stress tensor rotations
    return inv(T).dot(C).dot(TP)

def rotateCompliance(S, theta):
    '''Rotate the 2D compliance tensor S counterclockwise by angle theta.'''
    # Define rotation matrix T and T'
    T = rotaT(theta)
    TP = rotaTP(theta)

    # Stress tensor rotations
    return inv(TP).dot(S).dot(T)

def laminateStiffness(C, thetas, ts):
    '''Return the stiffness of a laminate given a single input C aligned with the
    fiber orientation of the base material, an array of angles corresponding to the
    laminate stack and an array of thicknesses ts corresponding to each lamina.'''
    return sum([rotateStiffness(C,theta)*t for theta,t in zip(thetas,ts)])/sum(ts)

def laminateCompliance(CLam):
    return inv(CLam)

def shortfiberstiffness(Em, Ef, V, s, vm):
    '''Calculates the stiffness of a short fiber composite using the modified shear lag model.
    Inputs are stiffness of fiber and matrix (Ef and Em), Volume fraction of fiber, fiber
    aspect ratio (s) and matrix Poisson's ratio vm'''
    n = np.sqrt(2*Em/(Ef*(1+vm)*np.log(1/V)))
    a = np.cosh(n*s)
    Emprime = (Ef*(1-(1/a)) + Em)
    angle = n*s*180/(np.pi)
    E = V*Ef*(1-((Ef-Emprime)*np.tanh(n*s))/(Ef*n*s)) + (1-V)*Em
    return E

def isLaminateBalanced(S, plot):
    '''Determine whether a laminate is balanced by inputting a compliance matrix S.
    Plot is True or False and will plot the tension-shear interaction ratios'''

    # Define angles between 0-90 degrees
    ths = [x*pi/180 for x in range(91)]

    # Find tension-shear interaction ratios
    nxyx = []; nxyy = []
    for th in ths:
      SLR = rotateCompliance(S, th)
      nxyx += [SLR[0][2]*SLR[0][0]]
      nxyy += [SLR[1][2]*SLR[1][1]]

    # Plot the results
    if plot:
        fig,axs = plt.subplots(nrows=2, ncols=1)
        axs[0].plot(ths, nxyx)
        axs[1].plot(ths, nxyy)
        axs[1].set_xlabel(r'Angle $\theta$')
        axs[0].set_ylabel(r'$\eta_{xyx}$')
        axs[1].set_ylabel(r'$\eta_{xyy}$')

    # Return balanced or not
    return all(x==0 for x in nxyx) and all(x==0 for x in nxyy)

def Thermal_strain(e1,e2,g12, alphm, alphf, Em, Ef, vm, vf, f, delT):
    '''Calculates the thermal strain induced in a composite'''
    alph11 = (alphf*f*Ef + alphm*(1-f)*Em)/(f*Ef + (1-f)*Em)
    alph22 = alphf*f*(1+vf) + alphm*(1-f)*(1+vm) - alph11*(f*vf + (1-f)*vm)
    strain = np.zeros((3,1))
    strain[0,0] = e1-alph11*delT
    strain[1,0] = e2-alph22*delT
    strain[2,0] = g12
#    print(alph11, alph22)

def ShearModulus(Gm, Gf, V):
    '''Returns the Shear modulus of a lamina by Halpin-Tsai formulation'''
    n = (Gf/Gm -1)/ (Gf/Gm + 1)
    G12 = Gm*(1 + n*V)/(1 - n*V)
    return (G12)

def Compressivestrength(G12, tau12, phi):
    '''Returns the compressive strength of a laminate when plastic microbuckling is the
    dominant failure mechanism. Takes an input shear modulus, shear strength and the
    imperfection angle '''
    sig_com = 1/((1/G12) + (phi/tau12))
    return (sig_com)

def Weibullfailurestrength(Pf,L,L0,sig0):
    '''Calculates the failure strength for a given probablity of failure using Weibull Theory.
    Takes an input probability of failure, Length of the fiber, Reference length of the fiber and the reference
    strength.'''
    Failure_strength = sig0*np.exp((np.log(np.log(1/(1-Pf))) -np.log(L/L0))/m)
    return (Failure_strength)


def maxStressFail(stress, s1, s2, s12):
    '''Determine whether a composite will fail under a max stress condition.
    Takes an input stress in 2D (aligned with the fiber direction) and strengths
    in the 1, 2, and 12 direction. Will return true or false.'''
    return stress[0]>s1 or stress[1]>s2 or stress[2]>s12


def tsaiHillFail(stress, s1, s2, s12):
    '''Determine whether a composite will fail under a Tsai-Hill condition.
    Takes an input stress in 2D (aligned with the fiber direction) and strengths
    in the 1, 2, and 12 direction. Will return true or false.'''
    return (stress[0]/s1)**2+(stress[1]/s2)**2-stress[0]*stress[1]/s1**2+(stress[2]/s12)**2 > 1


def laminaFail(stress, th, s1, s2, s12, criterion='Tsai-Hill'):
    '''Determine whether an arbitrary load will cause failure. Takes an input stress
    in 2D, an angle th, strengths in the 1, 2 and 12 direction, and the criterion to
    be used. Returns true or false.'''
    # Rotate stress
    s = rotateStress(stress,th)

    # Determine failure
    if criterion == 'Tsai-Hill':
        return tsaiHillFail(s, s1, s2, s12)
    elif criterion == 'Max Stress':
        return maxStressFail(s, s1, s2, s12)